In [5]:
import numpy as np
import tensorflow as tf

# Load and preprocess the text
text = open('input_text.txt', 'r').read()
vocab = sorted(set(text))
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = np.array(vocab)
text_as_int = np.array([char_to_index[c] for c in text])

# Create training examples and targets
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 1  # Set batch size to 1 for text generation

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Restore the latest checkpoint if available
checkpoint_dir = './training_checkpoints'
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if (latest_checkpoint is not None):
    model.load_weights(latest_checkpoint)
    model.build(tf.TensorShape([1, None]))  # Build the model with batch size 1

# Function to generate text
def generate_text(model, start_string, num_generate=100, temperature=0.1):
    # Ensure start_string contains only known characters
    for char in start_string:
        if char not in char_to_index:
            raise ValueError(f"Character '{char}' not found in char_to_index dictionary.")

    input_eval = [char_to_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(index_to_char[predicted_id])
    return (start_string + ''.join(text_generated))

# Find a suitable start string
suitable_start_string = ''.join([char for char in "The sun is like the superstar of our solar system! It's the big, glowing ball of light that's right at the center. Imagine a giant, flaming ball that's so bright you can't even look at it without squinting!" if char in char_to_index])

# Generate text
generated_text = generate_text(model, start_string=suitable_start_string, num_generate=100, temperature=0.1)
print(generated_text)


The sun is lie the superstar of our solar sstem! Its the big, gloing ball of light thats right at the center. Imagine a giant, flaming ball thats so bright ou cant even loo at it ithout squinting!PElxsMaQFntRt:?:MVdm,iTLAeRFCSBn.Nhi
mu!T hDu,nH-vTEs:BNrTn;rTuEM-VQ.?tlDZgdnsmMcH?xvxD-rHEQD MdFt-h


initially num generate value was 1000 temperature changed from 0.7 to 0.1